# Importing important modules and setting up the project

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB

from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import xgboost

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fare-classification/meta_data.csv
/kaggle/input/my-sample-data/train.csv
/kaggle/input/my-sample-data/test.csv


Using TensorFlow backend.


Reading the data from files

In [2]:
train = pd.read_csv('../input/my-sample-data/train.csv')
test = pd.read_csv('../input/my-sample-data/test.csv')

In [3]:
train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.0000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.0000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.0000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.6638,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct


In [4]:
predictor_cols = ['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup', 'fare']
train_X = train[predictor_cols]
test_X = test[predictor_cols]

#train_y.fillna(train_y.mean(), inplace=True)
train_X.fillna(train_X.mean(), inplace=True)
test_X.fillna(test_X.mean(), inplace=True)
train['num_label'] = False
train['direct_distance'] = 0.0
test['direct_distance'] = 0.0

for index, row in train.iterrows():
    train.at[index,'direct_distance'] = ((((row['drop_lon'] - row['pick_lon'])**2)+((row['drop_lat'] - row['pick_lat'])**2))**0.5)*111
    
for index, row in test.iterrows():
    test.at[index,'direct_distance'] = ((((row['drop_lon'] - row['pick_lon'])**2)+((row['drop_lat'] - row['pick_lat'])**2))**0.5)*111

for index, row in train.iterrows():
    if(row['label'] == "correct"):
        train.at[index, 'num_label'] = 1
    else:
        train.at[index, 'num_label'] = 0

predictor_cols = ['additional_fare', 'direct_distance', 'duration', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup', 'fare']
train_X_ = train[predictor_cols]
test_X = test[predictor_cols]
train_X_.fillna(train_X.mean(), inplace=True)
test_X.fillna(test_X.mean(), inplace=True)

train_y_ = train.num_label
#train_y = train['label']

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [5]:
train_X_.head()

,additional_fare,direct_distance,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare
0,10.500000,5.091511,834.000000,56.000000,0.000000,64.000000,270.32
1,10.500000,3.163550,791.000000,47.000000,0.000000,134.000000,197.85
2,10.500000,6.329083,1087.000000,80.000000,0.000000,61.000000,301.64
3,10.500000,0.866429,598.000000,271.000000,15.663800,68.000000,82.30
4,13.719651,8.182580,1702.858077,629.074231,32.057666,112.466832,358.39


In [6]:
train_X, val_X, train_y, val_y = train_test_split(train_X_, train_y_, test_size = 0.2, train_size = 0.8, random_state = 0)

In [7]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    print('Model Performance')
    print('Accuracy:%f'%accuracy)
    print("Mean cross validation score:%f"%score.mean())
    print('Mean Squared Error : %.4f' % mse)
    print('Root MSE : %.4f' % rmse)

Random forest model

In [8]:
my_random_forest_model = RandomForestRegressor()
my_random_forest_model.fit(train_X, train_y)

evaluate(my_random_forest_model, val_X, val_y)

Model Performance
Accuracy:0.952270
Mean cross validation score:0.484992
Mean Squared Error : 0.0386
Root MSE : 0.1965


Decision tree model

In [9]:
my_decision_tree_model = DecisionTreeRegressor()
my_decision_tree_model.fit(train_X, train_y)

evaluate(my_decision_tree_model, val_X, val_y)

Model Performance
Accuracy:0.923458
Mean cross validation score:-0.003552
Mean Squared Error : 0.0765
Root MSE : 0.2767


In [10]:
Ada_boost_model = AdaBoostRegressor()
Ada_boost_model.fit(train_X, train_y)

evaluate(Ada_boost_model, val_X, val_y)

Model Performance
Accuracy:0.925495
Mean cross validation score:0.045078
Mean Squared Error : 0.0762
Root MSE : 0.2760


In [11]:
#def create_baseline():
model = Sequential()
model.add(Dense(7, activation='relu'))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(7, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(1, activation='softmax'))
#model.add(Dense(1, activation='sigmoid'))
#model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Accuracy()])
#tf.keras.metrics.BinaryAccuracy()
#adam,sgd
#return model

#model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

#estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=10, verbose=0)
#kfold = StratifiedKFold(n_splits=10, shuffle=True)
#results = cross_val_score(estimator, train_X, train_y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
model.fit(train_X_.values, train_y_.values,
          batch_size=10,
          epochs=30,
          validation_split = 0.5,
          verbose = 1,
          shuffle=True)

#evaluate(model, val_X, val_y)

Train on 8588 samples, validate on 8588 samples
Epoch 1/30
8588/8588 [==============================] - 4s 416us/step - loss: 0.1164 - mean_squared_error: 0.1196 - accuracy: 0.8804 - val_loss: 0.0793 - val_mean_squared_error: 0.1037 - val_accuracy: 0.8963
Epoch 2/30
8588/8588 [==============================] - 3s 322us/step - loss: 0.1164 - mean_squared_error: 0.1008 - accuracy: 0.8992 - val_loss: 0.0793 - val_mean_squared_error: 0.1001 - val_accuracy: 0.8999
Epoch 3/30
8588/8588 [==============================] - 3s 345us/step - loss: 0.1164 - mean_squared_error: 0.0999 - accuracy: 0.9001 - val_loss: 0.0793 - val_mean_squared_error: 0.0992 - val_accuracy: 0.9008
Epoch 4/30
8588/8588 [==============================] - 3s 355us/step - loss: 0.1164 - mean_squared_error: 0.0991 - accuracy: 0.9009 - val_loss: 0.0793 - val_mean_squared_error: 0.0989 - val_accuracy: 0.9011
Epoch 5/30
8588/8588 [==============================] - 3s 358us/step - loss: 0.1164 - mean_squared_error: 0.0989 - accu

In [17]:
xg_model = xgboost.XGBClassifier(base_score=0.1, booster= None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.4, gpu_id=0, importance_type='gain',
              interaction_constraints=None, learning_rate=0.01,
              max_delta_step=0, max_depth=80, min_child_weight=7, missing=None,
              monotone_constraints=None, n_estimators=1000, n_jobs=6, nthread=6,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=False,
              subsample=0.8,tree_method='hist', validate_parameters=False,
              verbosity=1)
#xg_model = xgboost.XGBClassifier()
#base_score 0.5 -> 0.1
#score=cross_val_score(xg_model, train_X, train_y, cv=10)
xg_model.fit(train_X, train_y)


XGBClassifier(base_score=0.1, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0.4, gpu_id=0, importance_type='gain',
              interaction_constraints=None, learning_rate=0.01,
              max_delta_step=0, max_depth=80, min_child_weight=7, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=6, nthread=6,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=False,
              subsample=0.8, tree_method='hist', validate_parameters=False,
              verbosity=1)

In [18]:
def xg_evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    #mse = mean_squared_error(test_labels, predictions)
    #mae = mean_absolute_error(test_labels, predictions)
    #rmse = np.sqrt(mse)
    #score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    #print('Model Performance')
    print('Accuracy:%f'%accuracy)
    #print("Mean cross validation score:%f"%score.mean())
    #print('Mean Squared Error : %.4f' % mse)
    #print('Root MSE : %.4f' % rmse)
xg_evaluate(xg_model, val_X, val_y)

Accuracy:0.951688


In [26]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=7)
knn_classifier.fit(train_X, train_y)

def knn_evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    #mse = mean_squared_error(test_labels, predictions)
    #mae = mean_absolute_error(test_labels, predictions)
    #rmse = np.sqrt(mse)
    #score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    #print('Model Performance')
    print('Accuracy:%f'%accuracy)
    #print("Mean cross validation score:%f"%score.mean())
    #print('Mean Squared Error : %.4f' % mse)
    #print('Root MSE : %.4f' % rmse)
knn_evaluate(classifier, val_X, val_y)
'''
for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(train_X, train_y)
    knn_evaluate(classifier, val_X, val_y)'''

Accuracy:0.946740


'\nfor i in range(1, 40):\n    knn = KNeighborsClassifier(n_neighbors=i)\n    knn.fit(train_X, train_y)\n    knn_evaluate(classifier, val_X, val_y)'

In [27]:
predicted_rf = my_random_forest_model.predict(test_X)
predicted_dt = my_decision_tree_model.predict(test_X)
predicted_ad = Ada_boost_model.predict(test_X)
predicted_xg = xg_model.predict(test_X)
predicted_NN = model.predict(test_X)
predicted_KNN = knn_classifier.predict(test_X)

pred_KNN = []
for x in range(len(predicted_KNN)):
    pred_KNN.append(bool(predicted_KNN[x]))

predicted_NN = np.array(pred_NN)

pred_NN = []
for x in range(len(predicted_NN)):
    pred_NN.append(bool(predicted_NN[x]))

predicted_NN = np.array(pred_NN)
    
pred_rf = []
for x in range(len(predicted_rf)):
    pred_rf.append(bool(predicted_rf[x]))
    
predicted_rf = np.array(pred_rf)
    
pred_dt = []
for x in range(len(predicted_dt)):
    pred_dt.append(bool(predicted_dt[x]))

predicted_dt = np.array(pred_dt)

pred_ad = []
for x in range(len(predicted_ad)):
    pred_ad.append(bool(predicted_ad[x]))

predicted_ad = np.array(pred_ad)

pred_xg = []
for x in range(len(predicted_xg)):
    pred_xg.append(bool(predicted_xg[x]))
    
predicted_xg = np.array(pred_xg)

output_rf = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_rf})

output_dt = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_dt})

output_adb = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_dt})

output_xg = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_xg})

output_NN = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_NN})

output_KNN = pd.DataFrame({'tripid': test.tripid,
                       'prediction': predicted_KNN})

output_rf_path = "submission_rf.csv"
output_dt_path = "submission_dt.csv"
output_adb_path = "submission_adb.csv"
output_xg_path = "submission_xg.csv"
output_NN_path = "submission_nn.csv"
output_KNN_path = "submission_knn.csv"

output_rf.to_csv(output_rf_path, index=False)
output_dt.to_csv(output_dt_path, index=False)
output_adb.to_csv(output_adb_path, index = False)
output_xg.to_csv(output_xg_path, index = False)
output_NN.to_csv(output_NN_path, index = False)
output_KNN.to_csv(output_KNN_path, index = False)